To view detailed visualizations and logs of the evaluations, please visit the [WandB dashboard](https://wandb.ai/david-spannagl/Evaluating_phi-1.5_for_Kotlin-code-completion) associated with this project.

In [ ]:
!pip install torch torchvision
!pip install transformers
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install accelerate -U
!pip install wandb

In [2]:
import os
import torch
import requests

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from typing import Callable

In [ ]:
import wandb
wandb.login()

In [4]:
base_model_id = "microsoft/phi-1_5"
github_base_url = "https://raw.githubusercontent.com/DaveS24/KotComplete/main/"

tuned_model_config = github_base_url + "model/adapter_config.json"
tuned_model_weights = github_base_url + "model/adapter_model.safetensors"
tuned_model_training_args = github_base_url + "model/training_args.bin"

tuned_model_path = "/content/model/pre-trained/"
model_log_dir = "/content/model/log/testing/"

kotlin_data_url = github_base_url + "data/Kotlin/test.jsonl"
codexglue_data_url = github_base_url + "data/CodeXGLUE/test.jsonl"
dataset_loader_url = github_base_url + "src/dataset_loader.py"

In [5]:
response_config = requests.get(tuned_model_config)
response_weights = requests.get(tuned_model_weights)
response_training_args = requests.get(tuned_model_training_args)

os.makedirs(tuned_model_path, exist_ok=True)

with open(tuned_model_path + "adapter_config.json", 'wb') as file:
    file.write(response_config.content)

with open(tuned_model_path + "adapter_model.safetensors", 'wb') as file:
    file.write(response_weights.content)

with open(tuned_model_path + "training_args.bin", 'wb') as file:
    file.write(response_training_args.content)

In [6]:
response_data_parser = requests.get(dataset_loader_url)
dataset_loader_code = response_data_parser.text

load_jsonl_from_url: Callable
create_and_tokenize_dataset: Callable
dataset_summary: Callable

exec(dataset_loader_code)

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["fc1", "fc2", "Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
base_phi_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)
base_phi_model = get_peft_model(base_phi_model, lora_config)
base_phi_model.print_trainable_parameters()

In [ ]:
tuned_phi_model = AutoModelForCausalLM.from_pretrained(tuned_model_path, quantization_config=bnb_config)
tuned_phi_model = get_peft_model(tuned_phi_model, lora_config)
tuned_phi_model.print_trainable_parameters()

In [ ]:
phi_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
phi_tokenizer.pad_token = phi_tokenizer.eos_token

In [11]:
kotlin_test_data = load_jsonl_from_url(kotlin_data_url, use_subset=True, subset_ratio=0.25)
kotlin_test_dataset = create_and_tokenize_dataset(kotlin_test_data, phi_tokenizer)

dataset_summary(kotlin_test_dataset)

Dataset Info
Number of samples: 3306
Column names: ['input_ids', 'attention_mask', 'labels']
Features: {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [12]:
codexglue_test_data = load_jsonl_from_url(codexglue_data_url, use_subset=True, subset_ratio=0.25)
codexglue_test_dataset = create_and_tokenize_dataset(codexglue_test_data, phi_tokenizer)

dataset_summary(codexglue_test_dataset)

Dataset Info
Number of samples: 5000
Column names: ['input_ids', 'attention_mask', 'labels']
Features: {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [13]:
base_training_args = TrainingArguments(
    output_dir=model_log_dir + 'base/output/',
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    disable_tqdm=True
)

base_trainer = Trainer(
    model=base_phi_model,
    args=base_training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=phi_tokenizer, mlm=False)
)

In [14]:
run_base_kotlin = wandb.init(project='Evaluating_phi-1.5_for_Kotlin-code-completion', name='base_kotlin_test', job_type='eval', anonymous='allow')

base_trainer.evaluate(eval_dataset=kotlin_test_dataset)

run_base_kotlin.finish()

wandb: Currently logged in as: david-spannagl. Use `wandb login --relogin` to force relogin


{'eval_loss': 5.300319671630859, 'eval_runtime': 139.9988, 'eval_samples_per_second': 23.614, 'eval_steps_per_second': 2.957}


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/loss,5.30032
eval/runtime,139.9988
eval/samples_per_second,23.614
eval/steps_per_second,2.957
train/global_step,0


In [15]:
run_base_codexglue = wandb.init(project='Evaluating_phi-1.5_for_Kotlin-code-completion', name='base_codexglue_test', job_type='eval', anonymous='allow')

base_trainer.evaluate(eval_dataset=codexglue_test_dataset)

run_base_codexglue.finish()

{'eval_loss': 3.007777690887451, 'eval_runtime': 738.6539, 'eval_samples_per_second': 6.769, 'eval_steps_per_second': 0.846}


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/loss,3.00778
eval/runtime,738.6539
eval/samples_per_second,6.769
eval/steps_per_second,0.846
train/global_step,0


In [16]:
tuned_training_args = TrainingArguments(
    output_dir=model_log_dir + 'tuned/output/',
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    disable_tqdm=True
)

tuned_trainer = Trainer(
    model=tuned_phi_model,
    args=tuned_training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=phi_tokenizer, mlm=False)
)

In [17]:
run_tuned_kotlin = wandb.init(project='Evaluating_phi-1.5_for_Kotlin-code-completion', name='tuned_kotlin_test', job_type='eval', anonymous='allow')

tuned_trainer.evaluate(eval_dataset=kotlin_test_dataset)

run_tuned_kotlin.finish()

{'eval_loss': 5.300319671630859, 'eval_runtime': 151.2423, 'eval_samples_per_second': 21.859, 'eval_steps_per_second': 2.737}


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/loss,5.30032
eval/runtime,151.2423
eval/samples_per_second,21.859
eval/steps_per_second,2.737
train/global_step,0


In [18]:
run_tuned_codexglue = wandb.init(project='Evaluating_phi-1.5_for_Kotlin-code-completion', name='tuned_codexglue_test', job_type='eval', anonymous='allow')

tuned_trainer.evaluate(eval_dataset=codexglue_test_dataset)

run_tuned_codexglue.finish()

{'eval_loss': 3.007777690887451, 'eval_runtime': 739.9081, 'eval_samples_per_second': 6.758, 'eval_steps_per_second': 0.845}


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/loss,3.00778
eval/runtime,739.9081
eval/samples_per_second,6.758
eval/steps_per_second,0.845
train/global_step,0
